In [1]:
import os
import numpy as np
import re
import string

In [2]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [4]:
FILE_UNIGRAMS = '1grams-3.txt'
FILE_TEXT = 'text_Chekhov'
N_WORDS = 192689044
TOL = 1e-20

In [5]:
EXCLUDE_SYMBOLS_STR = u''.join(['№', '«', 'ђ', '°', '±', '‚', 'ћ', '‰', '…', '»', 'ѓ', 'µ', '·', 'ґ', 'њ', 'ї', 'џ', 'є',
                                '‹', '‡', '†', '¶', 'ќ', '€', '“', 'ў', '§', '„', '”', '\ufeff', '’', 'љ', '›', '•', '—',
                                '‘', '\x7f', '\xad', '¤', '\xa0'])

### [Частоты НКРЯ](http://www.ruscorpora.ru/corpora-freq.html)

---

In [6]:
txt = open(FILE_TEXT, 'r').read()

In [7]:
txt

'— Приходили от Григорьевых за какой-то книгой, но я сказала, что вас нет дома. Почтальон принес газеты и два письма. Кстати, Евгений Петрович, я просила бы вас обратить ваше внимание на Сережу. Сегодня и третьего дня я заметила, что он курит. Когда я стала его усовещивать, то он, по обыкновению, заткнул уши и громко запел, чтобы заглушить мой голос. Евгений Петрович Быковский, прокурор окружного суда, только что вернувшийся из заседания и снимавший у себя в кабинете перчатки, поглядел на докладывавшую ему гувернантку и засмеялся. — Сережа курит... — пожал он плечами. — Воображаю себе этого карапуза с папиросой! Да ему сколько лет? — Семь лет. Вам кажется это несерьезным, но в его годы курение составляет вредную и дурную привычку, а дурные привычки следует искоренять в самом начале. — Совершенно верно. А где он берет табак? — У вас в столе. — Да? В таком случае пришлите его ко мне. По уходе гувернантки Быковский сел в кресло перед письменным столом, закрыл глаза и стал думать. Он рисов

In [8]:
frequencies = {}

with open(FILE_UNIGRAMS, 'r') as f:
    for line in f:
        split = line.strip().split('\t')
        try:
            freq, word = split
        except:
            print(split)
        frequencies[word] = float(freq) / N_WORDS

In [9]:
len(frequencies)

1054210

In [10]:
frequencies

{'Верхарнов': 1.5569125974801143e-08,
 'коротающим': 1.5569125974801143e-08,
 'Нелидову': 1.868295116976137e-07,
 'лядами': 1.5569125974801143e-08,
 'судейству': 2.5948543291335235e-08,
 'меланхоличной': 2.5948543291335235e-08,
 'греке': 6.227650389920457e-08,
 'сопровождающуюся': 5.708679524093752e-08,
 'Штоки': 3.1138251949602286e-08,
 'Уверь': 4.1517669266136374e-08,
 'сорокалетней': 4.774531965605683e-07,
 'междугороднюю': 1.5569125974801143e-08,
 'сосисочные': 5.189708658267047e-08,
 'даоса': 3.632796060786933e-08,
 'Сеченовский': 8.82250471905398e-08,
 'Прошко': 3.1138251949602286e-08,
 'зажурчала': 5.189708658267047e-08,
 'Ударная': 2.854339762046876e-07,
 'Активе': 1.5569125974801143e-08,
 '102,6': 2.5948543291335235e-08,
 'целительной': 1.9201922035588073e-07,
 'самодержицею': 2.5948543291335235e-08,
 'Баграта': 1.4012213377321026e-07,
 'миллионерша': 2.1277805498894894e-07,
 'напрашиваемся': 2.5948543291335235e-08,
 'измену': 3.539381304938126e-06,
 'зампрокурора': 9.86044645

In [11]:
frequencies['и']

0.03544554406528687

In [12]:
regex_puncts = re.compile('[%s]' % re.escape(string.punctuation))
regex_symbs = re.compile('[%s]' % re.escape(EXCLUDE_SYMBOLS_STR))
regex_digits = re.compile('[%s]' % re.escape(string.digits))
regex_spaces = re.compile('[%s]' % string.printable + string.whitespace)

In [13]:
txt_prep = regex_puncts.sub('', txt)

In [14]:
txt_prep = regex_symbs.sub('', txt_prep)

In [15]:
txt_prep = regex_digits.sub('', txt_prep)

In [16]:
txt_prep = regex_spaces.sub('', txt_prep)

In [17]:
txt_prep = txt_prep.strip().strip('\t').replace('\n', '').lower()

In [18]:
txt_prep

'приходили от григорьевых за какойто книгой но я сказала что вас нет дома почтальон принес газеты и два письма кстати евгений петрович я просила бы вас обратить ваше внимание на сережу сегодня и третьего дня я заметила что он курит когда я стала его усовещивать то он по обыкновению заткнул уши и громко запел чтобы заглушить мой голос евгений петрович быковский прокурор окружного суда только что вернувшийся из заседания и снимавший у себя в кабинете перчатки поглядел на докладывавшую ему гувернантку и засмеялся  сережа курит  пожал он плечами  воображаю себе этого карапуза с папиросой да ему сколько лет  семь лет вам кажется это несерьезным но в его годы курение составляет вредную и дурную привычку а дурные привычки следует искоренять в самом начале  совершенно верно а где он берет табак  у вас в столе  да в таком случае пришлите его ко мне по уходе гувернантки быковский сел в кресло перед письменным столом закрыл глаза и стал думать он рисовал в воображении своего сережу почемуто с гро

In [19]:
words = txt_prep.split()

print(len(words))

1275


In [20]:
frequencies_empirical = {}

for word in log_progress(words):
    if frequencies_empirical.get(word, None) is None:
        frequencies_empirical[word] = 1
    else:
        frequencies_empirical[word] += 1

VBox(children=(HTML(value=''), IntProgress(value=0, max=1275)))

In [22]:
def perplexity(docs):
    docs = np.asarray(docs)
    out_of_dict = []
    
    tmp_sum_docs = 0.0
    N = 0.0
    for doc in log_progress(docs):
        tmp_sum_words = 0.0
        for word in log_progress(words):
            freq = frequencies.get(word, TOL)
            freq_empirical = frequencies_empirical.get(word, 0.0)
            N += freq_empirical
            if freq == TOL:
                out_of_dict.append(word)
            n_wd = words[words == word]
            tmp_sum_words += freq_empirical * np.log(freq)
        tmp_sum_docs += tmp_sum_words
        
    return np.exp(-tmp_sum_docs / N), out_of_dict

In [23]:
perp, ood = perplexity([words])

print(perp)

VBox(children=(HTML(value=''), IntProgress(value=0, max=1)))

VBox(children=(HTML(value=''), IntProgress(value=0, max=1275)))

303.48272292641303


In [24]:
print(np.unique(ood))

['быковский' 'григорьевых' 'двухтрех' 'докладывавшую' 'евгений' 'евгения'
 'евгеньич' 'игнатий' 'игнатия' 'натальи' 'наталья' 'петрович' 'петровича'
 'семеновна' 'семеновны' 'сергей' 'сережа' 'сереже' 'сережу']
